# 📘 Türkçe Metinlerle TF-IDF ve LSTM Uygulaması
Bu notebook, metin temizleme, TF-IDF vektörleştirme ve LSTM ile duygu analizi adımlarını Türkçe örneklerle içerir.

In [1]:
!pip install tensorflow scikit-learn nltk

In [2]:

import re
import numpy as np
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Türkçe stopword'leri yükle
stop_words = set(stopwords.words("turkish"))

# Metin temizleme fonksiyonu
def temizle_ve_tokenize(metin):
    temiz_metin = re.sub(r"[^\w\s]", "", metin)
    temiz_metin = temiz_metin.lower()
    kelimeler = temiz_metin.split()
    temiz_kelimeler = [k for k in kelimeler if k not in stop_words]
    return " ".join(temiz_kelimeler)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:

yorumlar = [
    "Film çok güzeldi, bayıldım!",
    "Hiç beğenmedim, çok sıkıcıydı.",
    "Oyunculuk harikaydı ama senaryo zayıftı.",
    "Bu kadar kötü bir film izlememiştim.",
    "Görsel efektler başarılıydı."
]
etiketler = np.array([1, 0, 1, 0, 1])

temizlenmis_yorumlar = [temizle_ve_tokenize(y) for y in yorumlar]
print(temizlenmis_yorumlar)


['film güzeldi bayıldım', 'beğenmedim sıkıcıydı', 'oyunculuk harikaydı senaryo zayıftı', 'kadar kötü bir film izlememiştim', 'görsel efektler başarılıydı']


In [4]:

vektorizer = TfidfVectorizer()
X_tfidf = vektorizer.fit_transform(temizlenmis_yorumlar)

print("TF-IDF Özellikler:", vektorizer.get_feature_names_out())
print("TF-IDF Vektör Matrisi:", X_tfidf.toarray())


TF-IDF Özellikler: ['bayıldım' 'başarılıydı' 'beğenmedim' 'bir' 'efektler' 'film' 'görsel'
 'güzeldi' 'harikaydı' 'izlememiştim' 'kadar' 'kötü' 'oyunculuk' 'senaryo'
 'sıkıcıydı' 'zayıftı']
TF-IDF Vektör Matrisi: [[0.61418897 0.         0.         0.         0.         0.49552379
  0.         0.61418897 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.70710678 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.70710678 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.5        0.         0.         0.
  0.5        0.5        0.         0.5       ]
 [0.         0.         0.         0.46369322 0.         0.37410477
  0.         0.         0.         0.46369322 0.46369322 0.46369322
  0.         0.         0.         0.        ]
 [0.         0.57735027 0.         0.         0.57735027 0.
  0.57735027 0.         0.         

In [5]:

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(temizlenmis_yorumlar)
sequences = tokenizer.texts_to_sequences(temizlenmis_yorumlar)
X_pad = pad_sequences(sequences, maxlen=10)

print("Sayı dizileri:", sequences)
print("Pad edilmiş girişler:", X_pad)


Sayı dizileri: [[1, 2, 3], [4, 5], [6, 7, 8, 9], [10, 11, 12, 1, 13], [14, 15, 16]]
Pad edilmiş girişler: [[ 0  0  0  0  0  0  0  1  2  3]
 [ 0  0  0  0  0  0  0  0  4  5]
 [ 0  0  0  0  0  0  6  7  8  9]
 [ 0  0  0  0  0 10 11 12  1 13]
 [ 0  0  0  0  0  0  0 14 15 16]]


In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32, input_length=10))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_pad, etiketler, epochs=10)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4000 - loss: 0.6938
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.6000 - loss: 0.6909
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.8000 - loss: 0.6880
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.6000 - loss: 0.6852
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6000 - loss: 0.6824
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6000 - loss: 0.6795
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6000 - loss: 0.6766
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6000 - loss: 0.6737
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.6000 - loss: 0.6706
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.6000 - loss: 0.6675


In [ ]:
import